<a href="https://colab.research.google.com/github/ttv0708/DSTalent/blob/main/CSC14118_Spark_Coding_Exercises_CQ22_21_(empty).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java and Spark on Hadoop

In [15]:
# install java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.5.6/spark-3.5.6-bin-hadoop3.tgz
# unzip the spark file to the current folder
!tar xf spark-3.5.6-bin-hadoop3.tgz

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,024 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,747 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,056

In [16]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.6-bin-hadoop3"

# Create a SparkSession in Python

In [17]:
# start pyspark
!pip install findspark
import findspark
findspark.init()

In [18]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
          .appName("Spark APIs Exercises")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()

# Example 1: WordCount with Spark DataFrames and Spark RDDs




In [21]:
# Load the data
!git clone https://github.com/nnthaofit/CSC14118.git

Cloning into 'CSC14118'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 818.44 KiB | 4.68 MiB/s, done.
Resolving deltas: 100% (1/1), done.


### Spark DataFrame-based WordCount

In [ ]:
linesDF = spark.read.text("CSC14118/ppap.txt")
linesDF.show(linesDF.count(),truncate = False)

from pyspark.sql import functions as f
wordsDF = linesDF.withColumn("word", f.explode(f.split(f.col("value"), " ")))\
    .groupBy("word")\
    .count()\
    .sort("count", ascending = False)
wordsDF.show()

+----------------------------+
|value                       |
+----------------------------+
|ppap                        |
|i have a pen                |
|i have an apple             |
|ah apple pen                |
|i have a pen                |
|i have a pineapple          |
|ah pineapple pen            |
|ppap pen pineapple apple pen|
+----------------------------+

+---------+-----+
|     word|count|
+---------+-----+
|      pen|    6|
|     have|    4|
|        i|    4|
|    apple|    3|
|pineapple|    3|
|        a|    3|
|     ppap|    2|
|       ah|    2|
|       an|    1|
+---------+-----+



###RDD-based WordCount

In [ ]:
linesRdd = spark.sparkContext.textFile("CSC14118/ppap.txt")
wordsRdd = linesRdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b)\
    .sortBy(lambda pair:-1*pair[1])
wordsRdd.collect()

[('pen', 6),
 ('i', 4),
 ('have', 4),
 ('a', 3),
 ('apple', 3),
 ('pineapple', 3),
 ('ppap', 2),
 ('ah', 2),
 ('an', 1)]

# Exercise 1: Data query with Spark DataFrame

In [ ]:
# clone the example data files from GitHub to Drive
!git clone https://github.com/nnthaofit/CSC14118.git

fatal: destination path 'CSC14118' already exists and is not an empty directory.


###0. Load the data file: movies.json

In [ ]:
df = spark.read.format("json").load("CSC14118/movies.json")
df.show()

+----------------+--------------------+--------------------+----+
|            cast|              genres|               title|year|
+----------------+--------------------+--------------------+----+
|              []|                  []|After Dark in Cen...|1900|
|              []|                  []|Boarding School G...|1900|
|              []|                  []|Buffalo Bill's Wi...|1900|
|              []|                  []|              Caught|1900|
|              []|                  []|Clowns Spinning Hats|1900|
|              []|[Short, Documentary]|Capture of Boer B...|1900|
|              []|                  []|The Enchanted Dra...|1900|
|   [Paul Boyton]|                  []|   Feeding Sea Lions|1900|
|              []|            [Comedy]|How to Make a Fat...|1900|
|              []|                  []|     New Life Rescue|1900|
|              []|                  []|    New Morning Bath|1900|
|              []|                  []|Searching Ruins o...|1900|
|         

### 1a. Show the schema of DataFrame that stores the movies dataset.

In [ ]:
df.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



### 1b. Show the number of distinct movies in the dataset

In [ ]:
df.distinct().count()

28789

### 2. Count the number of movies released during the years 2012 and 2015 (included)

In [ ]:
df.where((df.year>=2012) & (df.year<=2015)).count()

1015

### 3. Show the year in which the number of movies released is highest. One highest year is enough

In [ ]:
from pyspark.sql import functions as f
cntMoviesByYear = df.groupBy("year").count().sort("count",ascending=False)
cntMoviesByYear.show(1)

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+
only showing top 1 row



In [ ]:
# cách ngắn gọn hơn
df.groupBy("year").count().orderBy("count", ascending=False).show(1)

+----+-----+
|year|count|
+----+-----+
|1919|  634|
+----+-----+
only showing top 1 row



### 4. Show the list of movies such that for each film, the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.

In [ ]:
df.filter((f.size(df["cast"])>=5) & (f.size(df["genres"])<=2)).show()

+--------------------+----------------+--------------------+----+
|                cast|          genres|               title|year|
+--------------------+----------------+--------------------+----+
|[Earle Foxe, Alie...|         [Drama]|  A Desperate Chance|1913|
|[Charlotte Burton...|         [Drama]|    The Archeologist|1914|
|[Charlotte Burton...|         [Drama]|At the Potter's W...|1914|
|[Herbert Tracey, ...|        [Comedy]|    Back to the Farm|1914|
|[Charlotte Burton...|              []|    The Beggar Child|1914|
|[William Garwood,...|              []|       Billy's Rival|1914|
|[B. Reeves Eason,...|         [Drama]| Break, Break, Break|1914|
|[Charlotte Burton...|              []|       The Butterfly|1914|
|[Charlotte Burton...|       [Western]|Calamity Anne's L...|1914|
|[Charlie Chaplin,...|        [Comedy]|    The Star Boarder|1914|
|[Sydney Ayres, Ja...|              []|A Story of Little...|1914|
|[Sydney Ayres, Pe...|              []|The Story of the ...|1914|
|[Charlott

### 5. Show the **movies** whose names are longest

In [ ]:
max_val = df.select(f.length("title")).agg({"length(title)":"max"}).collect()[0][0]
df.where(f.length("title")==max_val).show(truncate=False)

+----+------+--------------------------------------------------------------------------------------------------------------+----+
|cast|genres|title                                                                                                         |year|
+----+------+--------------------------------------------------------------------------------------------------------------+----+
|[]  |[]    |Cornell-Columbia-University of Pennsylvania Boat Race at Ithaca, N.Y., Showing Lehigh Valley Observation Train|1901|
+----+------+--------------------------------------------------------------------------------------------------------------+----+



### 6. Show the movies whose name contains the word “fighting” (case-insensitive).

In [ ]:
df.where(f.lower("title").like("%fighting%")).show()

+--------------------+---------------+--------------------+----+
|                cast|         genres|               title|year|
+--------------------+---------------+--------------------+----+
|[Bessie Love, Ann...|[Comedy, Drama]|  A Fighting Colleen|1919|
|[Blanche Sweet, R...|      [Western]|     Fighting Cressy|1919|
|[Harry T. Morey, ...|        [Drama]|    Fighting Destiny|1919|
|[Tom Mix, Teddy S...|      [Western]|   Fighting for Gold|1919|
|[Jack Perrin, Hoo...|      [Western]|  The Fighting Heart|1919|
|[Art Acord, Mildr...|      [Western]|   The Fighting Line|1919|
|[William Duncan, ...|       [Action]|  The Fighting Guide|1922|
|[Tom Mix, Patsy R...|      [Western]| The Fighting Streak|1922|
|[Richard Barthelm...|   [Historical]|  The Fighting Blade|1923|
|[Ernest Torrence,...|       [Comedy]| The Fighting Coward|1924|
|[Jack Hoxie, Hele...|      [Western]|       Fighting Fury|1924|
|[Pat O'Malley, Ma...|        [Drama]|The Fighting Adve...|1924|
|[Fred Thomson, Ha...|   

### 7. Show the list of distinct genres appearing in the dataset

In [ ]:
df.select(f.explode(df["genres"]).alias("genre")).select("genre").distinct().show()

+-------------+
|        genre|
+-------------+
|        Crime|
|      Romance|
|     Thriller|
|      Slasher|
|Found Footage|
|    Adventure|
|         Teen|
| Martial Arts|
|       Sports|
|        Drama|
|          War|
|  Documentary|
|       Family|
|      Fantasy|
|       Silent|
|     Disaster|
|        Legal|
|      Mystery|
| Supernatural|
|     Suspense|
+-------------+
only showing top 20 rows



### 8. List all movies in which the actor Harrison Ford has participated.

In [ ]:
df.withColumn("actor",f.explode(df["cast"])).where(f.lower(f.col("actor"))=="harrison ford").select(df.columns).show()

+--------------------+-----------------+--------------------+----+
|                cast|           genres|               title|year|
+--------------------+-----------------+--------------------+----+
|[Constance Talmad...|[Romance, Comedy]|Experimental Marr...|1919|
|[Constance Talmad...|         [Comedy]| Happiness a la Mode|1919|
|[Constance Talmad...|         [Comedy]|Romance and Arabella|1919|
|[Vivian Martin, H...|         [Comedy]|      The Third Kiss|1919|
|[Harrison Ford, C...|         [Comedy]|The Veiled Adventure|1919|
|[Constance Talmad...|         [Comedy]|          Who Cares?|1919|
|[Vivian Martin, H...|          [Drama]|You Never Saw Suc...|1919|
|[Norma Talmadge, ...|          [Drama]| The Wonderful Thing|1921|
|[Alma Rubens, Har...|        [Mystery]|      Find the Woman|1922|
|[Constance Talmad...|          [Drama]| The Primitive Lover|1922|
|[Norma Talmadge, ...| [Romance, Drama]|     Smilin' Through|1922|
|[Helen Jerome Edd...|          [Drama]|     When Love Comes|1

### 9. List all movies in which the actors/actresses whose names include the word “Lewis“ (case-insensitive) have participated.

In [ ]:
df.withColumn("actor",f.explode(df["cast"])).where(f.lower(f.col("actor")).like("%lewis%")).select(df.columns).show()

+--------------------+-----------+--------------------+----+
|                cast|     genres|               title|year|
+--------------------+-----------+--------------------+----+
|[Charlotte Burton...|         []|       The Butterfly|1914|
|[Pearl White, She...|    [Drama]|The Exploits of E...|1914|
|[Charlotte Burton...|   [Comedy]| Mein Lieber Katrina|1914|
|[Norma Talmadge, ...|    [Drama]|      Going Straight|1916|
|[Dorothy Gish, Ra...|    [Drama]|Gretchen the Gree...|1916|
|[Ben Lewis, Bessi...|  [Western]|     A Sister of Six|1916|
|[Gail Kane, Lewis...|    [Drama]| The Bride's Silence|1917|
|    [Mitchell Lewis]|    [Drama]|Nine-Tenths of th...|1918|
|[Mitchell Lewis, ...|    [Drama]|The Faith of the ...|1919|
|[Mary Pickford, R...|   [Comedy]|         The Hoodlum|1919|
|[Mitchell Lewis, ...|    [Drama]|Jacques of the Si...|1919|
|[Mitchell Lewis, ...|    [Drama]|The Last of His P...|1919|
|[Lewis Stone, Jan...|    [Drama]|        Man's Desire|1919|
|[Mary Miles Minte...|  

### 10. Show top five actors/actresses that have participated in most movies.

In [ ]:
df.select(f.explode(df["cast"]).alias("actor"), "title").groupBy("actor").count().sort(f.col('count'), ascending=False).show(5)

+----------------+-----+
|           actor|count|
+----------------+-----+
|    Harold Lloyd|  190|
|     Hoot Gibson|  142|
|      John Wayne|  136|
|Charles Starrett|  116|
|    Bebe Daniels|  103|
+----------------+-----+
only showing top 5 rows



#Exercise 2: RDD-based mainpulation


*   The data is already in one ore more RDDs.
*   You must not convert RDD to DF or use pure Python code.


### 1. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a palindrome (case-insensitive).

In [ ]:
s = "Dogma I am God"
rdd = spark.sparkContext.parallelize(s.lower(), 1).filter(lambda letter: letter != ' ')
rdd.collect()

['d', 'o', 'g', 'm', 'a', 'i', 'a', 'm', 'g', 'o', 'd']

In [ ]:
rdd_id = spark.sparkContext.parallelize(range(0, rdd.count()), 1)
rdd_id.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
rdd_a = rdd_id.zip(rdd)
rdd_a.collect()

[(0, 'd'),
 (1, 'o'),
 (2, 'g'),
 (3, 'm'),
 (4, 'a'),
 (5, 'i'),
 (6, 'a'),
 (7, 'm'),
 (8, 'g'),
 (9, 'o'),
 (10, 'd')]

In [ ]:
rdd_b = rdd_a.sortBy(lambda row: row[0]*-1)
rdd_b.collect()

[(10, 'd'),
 (9, 'o'),
 (8, 'g'),
 (7, 'm'),
 (6, 'a'),
 (5, 'i'),
 (4, 'a'),
 (3, 'm'),
 (2, 'g'),
 (1, 'o'),
 (0, 'd')]

In [ ]:
rdd_compare = rdd_a.zip(rdd_b)
rdd_compare.collect()

[((0, 'd'), (10, 'd')),
 ((1, 'o'), (9, 'o')),
 ((2, 'g'), (8, 'g')),
 ((3, 'm'), (7, 'm')),
 ((4, 'a'), (6, 'a')),
 ((5, 'i'), (5, 'i')),
 ((6, 'a'), (4, 'a')),
 ((7, 'm'), (3, 'm')),
 ((8, 'g'), (2, 'g')),
 ((9, 'o'), (1, 'o')),
 ((10, 'd'), (0, 'd'))]

In [ ]:
if (rdd_compare.filter(lambda row: row[0][1] != row[1][1]).count() == 0):
  print("Palindrome")
else:
  print("Not palindrome")

Palindrome


### 2. Consider a string s that includes only alphabetical letters and spaces. Check whether s is a pangram (case-insensitive).

In [ ]:
# s = "The quick brown fox jumps over the lazy dog"
s = "The man who passes the sentence should swing the sword"
rdd = spark.sparkContext.parallelize(s.lower(), 1).filter(lambda letter: letter != ' ')
rdd.collect()

['t',
 'h',
 'e',
 'm',
 'a',
 'n',
 'w',
 'h',
 'o',
 'p',
 'a',
 's',
 's',
 'e',
 's',
 't',
 'h',
 'e',
 's',
 'e',
 'n',
 't',
 'e',
 'n',
 'c',
 'e',
 's',
 'h',
 'o',
 'u',
 'l',
 'd',
 's',
 'w',
 'i',
 'n',
 'g',
 't',
 'h',
 'e',
 's',
 'w',
 'o',
 'r',
 'd']

In [ ]:
if rdd.distinct().count() == 26:
  print("Pangram")
else:
  print("Not pangram")

Not pangram


#Exercise 3: Frequent patterns and association rules mining

### 0. Load the data file: foodmart.csv


*  A record is a tuple of binary values {0, 1}, each of which denotes the presence of an item (1: bought, 0: not bought).



In [ ]:
!git clone https://github.com/nnthaofit/CSC14118.git
df = spark.read.csv("CSC14118/foodmart.csv", header=True, inferSchema = True)
df.show()

fatal: destination path 'CSC14118' already exists and is not an empty directory.
+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+-

### 1. Convert the given data to the format required by Spark MLlib FPGrowth.

In [ ]:
from pyspark.sql.functions import col, array, lit, when
from pyspark.sql import functions as F

In [ ]:
# Lấy tên tất cả các cột
columns = df.columns

In [ ]:
# Chuyển mỗi hàng thành danh sách các mặt hàng có giá trị == 1
df_array = df.select(
    F.array([
        F.when(F.col(col) == 1, F.lit(col)).otherwise(F.lit(None))
        for col in columns
    ]).alias("items")
)
df_array.show()

+--------------------+
|               items|
+--------------------+
|[Acetominifen, NU...|
|[Acetominifen, NU...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
|[NULL, NULL, NULL...|
+--------------------+
only showing top 20 rows



In [ ]:
# Loại bỏ các NULL trong mảng (chỉ giữ các mục đã mua)
df_clean = df_array.selectExpr("filter(items, x -> x is not null) as items")
df_clean.show(truncate=False)

+---------------------------------------------------------------------+
|items                                                                |
+---------------------------------------------------------------------+
|[Acetominifen, Cheese, Home Magazines, Shampoo]                      |
|[Acetominifen, Cheese, Hard Candy, Milk, Pot Scrubbers, Rice]        |
|[Coffee, Deli Salads]                                                |
|[Eggs, Gum, Milk, Soup]                                              |
|[Cheese, Dried Fruit, Frozen Chicken, Plastic Utensils]              |
|[Shampoo]                                                            |
|[Milk, Paper Wipes, Waffles]                                         |
|[Donuts, Dried Fruit, Frozen Chicken]                                |
|[Cooking Oil, Hamburger, Maps, Popsicles]                            |
|[Cheese, Cooking Oil, Dips, Preserves, TV Dinner]                    |
|[Nasal Sprays]                                                 

###2.	Apply Spark MLlib FPGrowth to the formatted data. Mine the set of frequent patterns with the minimum support of 0.1. Mine the set of association rules with the minimum confidence of 0.9.

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
# Khởi tạo FP-Growth model
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.1)
model = fpGrowth.fit(df_clean)

In [ ]:
# Tập mục phổ biến
frequent_items = model.freqItemsets
frequent_items.sort("freq", ascending=False).show(truncate=False)

+-------------------+----+
|items              |freq|
+-------------------+----+
|[Cheese]           |285 |
|[Soup]             |280 |
|[Dried Fruit]      |256 |
|[Cookies]          |238 |
|[Wine]             |185 |
|[Preserves]        |185 |
|[Frozen Vegetables]|173 |
|[Canned Vegetables]|172 |
|[Nuts]             |170 |
|[Paper Wipes]      |164 |
|[Milk]             |164 |
|[Chocolate Candy]  |152 |
|[Chips]            |145 |
|[Sliced Bread]     |137 |
|[Cooking Oil]      |134 |
|[Muffins]          |132 |
|[Waffles]          |132 |
|[Eggs]             |128 |
|[Dips]             |127 |
|[Cereal]           |125 |
+-------------------+----+
only showing top 20 rows



In [ ]:
# Luật kết hợp
association_rules = model.associationRules
association_rules.show(truncate=False)

+-------------+-------------------+-------------------+------------------+--------------------+
|antecedent   |consequent         |confidence         |lift              |support             |
+-------------+-------------------+-------------------+------------------+--------------------+
|[Dried Fruit]|[Soup]             |0.140625           |1.068247767857143 |0.01692524682651622 |
|[Dried Fruit]|[Cheese]           |0.125              |0.9328947368421052|0.015044663845792195|
|[Dried Fruit]|[Wine]             |0.109375           |1.2575168918918918|0.013164080865068171|
|[Dried Fruit]|[Cookies]          |0.1171875          |1.0473017331932772|0.014104372355430184|
|[Milk]       |[Soup]             |0.1524390243902439 |1.1579921602787457|0.011753643629525154|
|[Lightbulbs] |[Dried Fruit]      |0.2                |1.6617187500000001|0.011283497884344146|
|[Wine]       |[Dried Fruit]      |0.15135135135135136|1.257516891891892 |0.013164080865068171|
|[Wine]       |[Soup]             |0.151

In [ ]:
# Dự đoán mục tiếp theo có thể mua dựa trên các luật
predictions = model.transform(df_clean)
predictions.select("items", "prediction").show(truncate=False)

+---------------------------------------------------------------------+----------------------------------------------------------------------------------+
|items                                                                |prediction                                                                        |
+---------------------------------------------------------------------+----------------------------------------------------------------------------------+
|[Acetominifen, Cheese, Home Magazines, Shampoo]                      |[Dried Fruit, Preserves, Soup]                                                    |
|[Acetominifen, Cheese, Hard Candy, Milk, Pot Scrubbers, Rice]        |[Soup, Dried Fruit, Preserves]                                                    |
|[Coffee, Deli Salads]                                                |[]                                                                                |
|[Eggs, Gum, Milk, Soup]                                              

#Exercise 4: Classification

###0. Load the data file: mushroom.csv
*   The data represents a collection of mushroom species.
*   There are 8124 examples, each of which has 22 attributes and it is categorized into either “edible” (e) or “poisonous” (p)


In [ ]:
df = spark.read.csv("CSC14118/mushrooms.csv", header=True, inferSchema=True)
df.show()

+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|class|cap-shape|cap-surface|cap-color|bruises|odor|gill-attachment|gill-spacing|gill-size|gill-color|stalk-shape|stalk-root|stalk-surface-above-ring|stalk-surface-below-ring|stalk-color-above-ring|stalk-color-below-ring|veil-type|veil-color|ring-number|ring-type|spore-print-color|population|habitat|
+-----+---------+-----------+---------+-------+----+---------------+------------+---------+----------+-----------+----------+------------------------+------------------------+----------------------+----------------------+---------+----------+-----------+---------+-----------------+----------+-------+
|    p|        x|          s|        n|      t|   p|              f|           c|        n|   

### 1.	Prepare the train and test sets following the ratio 8:2

In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

### 2. Fit a decision tree model on the training set, using Spark MLlib DecisionTreeClassifier with default parameters

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

# Lấy tên các cột
label_col = 'class'  # Tên cột phân loại: 'e' hoặc 'p'
feature_cols = [col for col in df.columns if col != label_col]

# Mã hóa label
label_indexer = StringIndexer(inputCol=label_col, outputCol="label")

# Mã hóa các feature dạng chuỗi
feature_indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="keep")
    for col in feature_cols
]

# Tập hợp các đặc trưng lại thành 1 vector
assembler = VectorAssembler(
    inputCols=[f"{col}_index" for col in feature_cols],
    outputCol="features"
)

# Decision Tree
dt = DecisionTreeClassifier(featuresCol="features", labelCol="label")

# Pipeline
dt_pipeline = Pipeline(stages=[label_indexer] + feature_indexers + [assembler, dt])

# Huấn luyện
dt_model = dt_pipeline.fit(train_data)


### 3. Fit a random forest model on the training set, using Spark MLlib RandomForestClassification with default parameters

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Random Forest
rf = RandomForestClassifier(featuresCol="features", labelCol="label")

# Pipeline
rf_pipeline = Pipeline(stages=[label_indexer] + feature_indexers + [assembler, rf])

# Huấn luyện
rf_model = rf_pipeline.fit(train_data)

### 4. Evaluate the two models on the same test set using the following metrics: areaUnderROC and areaUnderPR

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Dự đoán trên tập test
dt_predictions = dt_model.transform(test_data)
rf_predictions = rf_model.transform(test_data)

evaluator_roc = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
evaluator_pr = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderPR")

# Đánh giá Decision Tree
dt_auc = evaluator_roc.evaluate(dt_predictions)
dt_pr = evaluator_pr.evaluate(dt_predictions)

# Đánh giá Random Forest
rf_auc = evaluator_roc.evaluate(rf_predictions)
rf_pr = evaluator_pr.evaluate(rf_predictions)

print(f"Decision Tree - AUC: {dt_auc:.4f}, PR: {dt_pr:.4f}")
print(f"Random Forest - AUC: {rf_auc:.4f}, PR: {rf_pr:.4f}")

Decision Tree - AUC: 0.9948, PR: 0.9973
Random Forest - AUC: 1.0000, PR: 1.0000


###5. Chain the above steps into a single pipeline

# Exercise 5: Clustering

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MushroomClustering").getOrCreate()
df = spark.read.csv("CSC14118/mushrooms.csv", header=True, inferSchema=True)

# Xử lý: Mã hóa tất cả cột kiểu chuỗi
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_idx", handleInvalid="keep")
    for col in df.columns
]

# VectorAssembler cho toàn bộ cột đã mã hóa
assembler = VectorAssembler(
    inputCols=[f"{col}_idx" for col in df.columns],
    outputCol="features"
)

pipeline = Pipeline(stages=indexers + [assembler])
df_transformed = pipeline.fit(df).transform(df)


### 1.	Cluster the data by using Spark MLlib KMeans with k = 2, 3, and 5, using Euclidean distance and cosine distance

In [ ]:
from pyspark.ml.clustering import KMeans

def train_kmeans_models(df, ks, distance):
    models = {}
    for k in ks:
        kmeans = KMeans(featuresCol="features", predictionCol="prediction", k=k, distanceMeasure=distance, seed=42)
        model = kmeans.fit(df)
        models[(k, distance)] = model.transform(df)
    return models

ks = [2, 3, 5]
models_euclidean = train_kmeans_models(df_transformed, ks, "euclidean")
models_cosine = train_kmeans_models(df_transformed, ks, "cosine")

### 2. Evaluate each of the above clustering results using silhoutte score. Which configuration yeilds the best clustering?

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction", metricName="silhouette")

def evaluate_models(models):
    scores = {}
    for (k, dist), df_clustered in models.items():
        score = evaluator.evaluate(df_clustered)
        scores[(k, dist)] = score
    return scores

scores_euclidean = evaluate_models(models_euclidean)
scores_cosine = evaluate_models(models_cosine)

print("Silhouette Scores (Euclidean):")
for key, val in scores_euclidean.items():
    print(f"k={key[0]} → score={val:.4f}")

print("\nSilhouette Scores (Cosine):")
for key, val in scores_cosine.items():
    print(f"k={key[0]} → score={val:.4f}")

Silhouette Scores (Euclidean):
k=2 → score=0.2807
k=3 → score=0.3782
k=5 → score=0.3139

Silhouette Scores (Cosine):
k=2 → score=0.2758
k=3 → score=0.2663
k=5 → score=0.3051


###3. Chain the above steps into a single pipeline

In [ ]:
def full_pipeline(df, ks, distance):
    pipeline = Pipeline(stages=indexers + [assembler])
    df_transformed = pipeline.fit(df).transform(df)
    models = train_kmeans_models(df_transformed, ks, distance)
    scores = evaluate_models(models)
    return models, scores

models_euclidean, scores_euclidean = full_pipeline(df, ks, "euclidean")
models_cosine, scores_cosine = full_pipeline(df, ks, "cosine")

###4. For each clustering result obtained above, count the number of examples that belong to each of the three species.

In [ ]:
def count_clusters(models):
    for (k, dist), df_clustered in models.items():
        print(f"\n=== Clusters for k={k}, distance={dist} ===")
        df_clustered.groupBy("prediction").count().orderBy("prediction").show()

count_clusters(models_euclidean)
count_clusters(models_cosine)


=== Clusters for k=2, distance=euclidean ===
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 4745|
|         1| 3379|
+----------+-----+


=== Clusters for k=3, distance=euclidean ===
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 4593|
|         1| 1258|
|         2| 2273|
+----------+-----+


=== Clusters for k=5, distance=euclidean ===
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 1832|
|         1| 1860|
|         2| 1244|
|         3| 1553|
|         4| 1635|
+----------+-----+


=== Clusters for k=2, distance=cosine ===
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 2907|
|         1| 5217|
+----------+-----+


=== Clusters for k=3, distance=cosine ===
+----------+-----+
|prediction|count|
+----------+-----+
|         0| 2230|
|         1| 3431|
|         2| 2463|
+----------+-----+


=== Clusters for k=5, distance=cosine ===
+----------+-----+
|prediction|count|
+----------+-----+
|     

## Exercise 6: Network manipulation with Spark GraphFrames

###0. Load the data files: users.txt and followers.txt

In [1]:
# Nếu bạn dùng notebook trong môi trường Google Colab hoặc Jupyter
!pip install graphframes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.2 MB/s eta 0:00:00


In [19]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GraphFramesExample") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.3-spark3.1-s_2.12") \
    .getOrCreate()

from pyspark.sql.functions import col
from graphframes import GraphFrame


In [22]:
# Đọc file người dùng
vertices = spark.read.csv("CSC14118/users.txt", header=True, inferSchema=True)
vertices.show()

# Đọc file mối quan hệ
edges = spark.read.csv("CSC14118/followers.txt", header=True, inferSchema=True)
edges.show()

+---+-------------+---------------+
|  1|  BarackObama|   Barack Obama|
+---+-------------+---------------+
|  2|     ladygaga|Goddess of Love|
|  3|      jeresig|     John Resig|
|  4| justinbieber|  Justin Bieber|
|  6|matei_zaharia|  Matei Zaharia|
|  7|      odersky| Martin Odersky|
|  8|      anonsys|           NULL|
+---+-------------+---------------+

+---+
|2 1|
+---+
|4 1|
|1 2|
|6 3|
|7 3|
|7 6|
|6 7|
|3 7|
+---+



###1.	Construct a graph from the given data to demonstrate a tiny social network


In [23]:
g = GraphFrame(vertices, edges)
g.vertices.show()
g.edges.show()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


Py4JJavaError: An error occurred while calling o60.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4

###2.	Apply Graphs graphPageRank to the network to obtain a ranking list of users in terms of followers

###3. Find connected components on the graph, using Graphs connectedComponents or stronglyConnectedComponents